In [29]:
import os
import pandas as pd

In [31]:
instruction = "reasoned_fewshot"
preds_paths = [
    f"../results/pappa_dim1_{instruction}_text-davinci-003/raw_predictions.txt",
    f"../results/pappa_dim1_{instruction}_sh1_text-davinci-003/raw_predictions.txt",
    f"../results/pappa_dim1_{instruction}_sh2_text-davinci-003/raw_predictions.txt",
]
outdir = f"../results/pappa_dim1_{instruction}_majority_text-davinci-003"
os.makedirs(outdir, exist_ok=True)

# read predictions and save each in a column of a dataframe table
preds = []
for preds_path in preds_paths:
    with open(preds_path, "r") as f:
        # remove newline characters
        preds.append([line.rstrip() for line in f.readlines()])

# create dataframe
df = pd.DataFrame()
df["preds"] = preds[0]
df["preds_sh1"] = preds[1]
df["preds_sh2"] = preds[2]


# get the majority vote
def majority_vote(row):
    # get the most common prediction
    return row.value_counts().index[0]

df["majority_vote"] = df.apply(majority_vote, axis=1)

# retain only columns where the majority vote is different from one of the original predictions
df_non_unanimous = df[(df["majority_vote"] != df["preds"]) | (df["majority_vote"] != df["preds_sh1"]) | (df["majority_vote"] != df["preds_sh2"])]
print("Vote in not uninimous in", len(df_non_unanimous), "out of", len(df), "cases.")

# save majority vote predictions
with open(os.path.join(outdir, "raw_predictions.txt"), "w") as f:
    for pred in df["majority_vote"]:
        f.write(pred + "\n")

Vote in not uninimous in 78 out of 350 cases.
